# Investment Opportunities and Alignment

- how investments consistent with the Paris Agreement Long-term target look like
- the investments of today determine to a large extent the emissions of tomorrow

In [2]:
using CSV
using Statistics
using JSON

data = CSV.read("source/41560_2018_179_MOESM2_ESM-1.csv")

,Model,Region,Scenario,Variable
,String,String,String,String
1,AIM/CGE,World,CPol,Total energy investment
2,AIM/CGE,World,CPol,Total energy investment (supply side)
3,AIM/CGE,World,CPol,Low carbon investment
4,AIM/CGE,World,CPol,Low carbon investment (supply side)
5,AIM/CGE,World,CPol,Total_inv/GDP
6,AIM/CGE,World,CPol,LC_inv/GDP
7,AIM/CGE,World,CPol,Extraction and Conversion - Fossil Fuels
8,AIM/CGE,World,CPol,Electricity - Fossil Fuels w/o CCS
9,AIM/CGE,World,CPol,Hydrogen - Fossil


In [9]:
unique(data[!,:Variable])

31-element Array{String,1}:
 "Total energy investment"                                
 "Total energy investment (supply side)"                  
 "Low carbon investment"                                  
 "Low carbon investment (supply side)"                    
 "Total_inv/GDP"                                          
 "LC_inv/GDP"                                             
 "Extraction and Conversion - Fossil Fuels"               
 "Electricity - Fossil Fuels w/o CCS"                     
 "Hydrogen - Fossil"                                      
 "Electricity - Non-bio Renewables"                       
 "Hydrogen - Non-fossil"                                  
 "Extraction and Conversion - Bioenergy"                  
 "Extraction and Conversion - Nuclear"                    
 ⋮                                                        
 "Energy Supply|Electricity|Gas|w/o CCS"                  
 "Energy Supply|Electricity|Oil|w/ CCS"                   
 "Energy Supply|Electricity|

## Defining valid values

In [26]:
VARIABLES = (
  "Energy Efficiency",
  "CCS",
  "Electricity - T&D and Storage",
  "Extraction and Conversion - Nuclear",
  "Extraction and Conversion - Bioenergy",
  "Hydrogen - Non-fossil",
  "Electricity - Non-bio Renewables", # This will only be used to calculate other values. See getValues function
  "Energy Supply|Electricity|Solar",
  "Energy Supply|Electricity|Wind",
  "Hydrogen - Fossil",
  "Electricity - Fossil Fuels w/o CCS",
  "Extraction and Conversion - Fossil Fuels", # This will only be used to calculate other values. See getValues function
  "other renewables", # This is not in the data and will be calculated by "Electricity - Non-bio Renewables" minus "Energy Supply|Electricity|Solar" and "Energy Supply|Electricity|Wind". See getValues function
  "Coal", # This is not in the data and will be calculated by "Extraction and Conversion - Fossil Fuels" * 0.1. See getValues function
  "Oil and Gas" # This is not in the data and will be calculated by "Extraction and Conversion - Fossil Fuels" * 0.9. See getValues function
)

("Energy Efficiency", "CCS", "Electricity - T&D and Storage", "Extraction and Conversion - Nuclear", "Extraction and Conversion - Bioenergy", "Hydrogen - Non-fossil", "Electricity - Non-bio Renewables", "Energy Supply|Electricity|Solar", "Energy Supply|Electricity|Wind", "Hydrogen - Fossil", "Electricity - Fossil Fuels w/o CCS", "Extraction and Conversion - Fossil Fuels", "other renewables", "Coal", "Oil and Gas")

In [12]:
MODELS = (
  "AIM/CGE",
  "IMAGE",
  "MESSAGEix-GLOBIOM",
  "POLES",
  "REMIND-MAgPIE"
)

("AIM/CGE", "IMAGE", "MESSAGEix-GLOBIOM", "POLES", "REMIND-MAgPIE")

In [33]:
REGIONS = (
  "World",
  "CHN",
  "IND",
  "USA",
  "R5OECD90+EU",
  "R5REF",
  "R5ASIA",
  "R5MAF",
  "R5LAM",
  "EU"
)

("World", "CHN", "IND", "USA", "R5OECD90+EU", "R5REF", "R5ASIA", "R5MAF", "R5LAM", "EU")

In [14]:
SCENARIOS = (
  "1.5C",
  "CPol",
  "NDC",
  "2C"
)

("1.5C", "CPol", "NDC", "2C")

The `in` function checks if the first parameter is in second parameter.

## Filtering runs

After defining the valid values, we filter the runs by checking if each attributes is present in the value tuples.

In [15]:
function included(d)
    return in(d[:Variable], VARIABLES) && in(d[:Model], MODELS) && in(d[:Region], REGIONS) && in(d[:Scenario], SCENARIOS)
end

datum = filter(included, data)

,Model,Region,Scenario,Variable,Unit
,String,String,String,String,String
1,AIM/CGE,World,CPol,Extraction and Conversion - Fossil Fuels,Billion US$2015/yr
2,AIM/CGE,World,CPol,Electricity - Fossil Fuels w/o CCS,Billion US$2015/yr
3,AIM/CGE,World,CPol,Hydrogen - Fossil,Billion US$2015/yr
4,AIM/CGE,World,CPol,Electricity - Non-bio Renewables,Billion US$2015/yr
5,AIM/CGE,World,CPol,Hydrogen - Non-fossil,Billion US$2015/yr
6,AIM/CGE,World,CPol,Extraction and Conversion - Bioenergy,Billion US$2015/yr
7,AIM/CGE,World,CPol,Extraction and Conversion - Nuclear,Billion US$2015/yr
8,AIM/CGE,World,CPol,Electricity - T&D and Storage,Billion US$2015/yr
9,AIM/CGE,World,CPol,CCS,Billion US$2015/yr


## Prepare functions

### Summing up values

Each year has it own column in the source data and we want to sum it up as one value.
Additionally, we need to parse the value and convert it from German notation to English.

In [16]:
YEARS = (2015, 2020, 2025, 2030, 2035, 2040, 2045, 2050, 2055, 2060, 2065, 2070, 2075, 2080, 2085, 2090, 2095, 2100)

(2015, 2020, 2025, 2030, 2035, 2040, 2045, 2050, 2055, 2060, 2065, 2070, 2075, 2080, 2085, 2090, 2095, 2100)

To get a value in a DataFrame we use [x, y], where `x` is the row number and `y` the cell id. This would usually look something like `[1, :Region]`, but in this special case the id of the column is a `Symbol` with the name of the year. So we use `Symbol(year)`.

In [17]:
function sumYears(row)
    sum = 0
    for year in YEARS
        raw = row[1, Symbol(year)]
        value = parse(Float64, replace(raw, "," => "."))
        sum += value
    end
    return round(sum, digits=3)
end

sumYears (generic function with 1 method)

### GetValues
This function finds the rows in the DataFrame, gets every value and calculates the average.

First, we create an empty Dictionary with `Strings` as keys and `Float64` as numbers.
We loop over the `MODELS` and search for each row with the current model.
If we found one, we summarise the years and add the result to the Dictionary

After the loop, we get the values of the dictionary and calculate the mean value.

In [27]:
function getValues(scenario, variable, region)
    dict = Dict{String, Float64}()
    for model in MODELS
        # In some cases we need to calculate the values
        if (variable === "other renewables")
            renewables = findRow(scenario, "Electricity - Non-bio Renewables", region, model)
            wind = findRow(scenario, "Energy Supply|Electricity|Wind", region, model)
            solar = findRow(scenario, "Energy Supply|Electricity|Solar", region, model)
            
            if (size(renewables, 1) > 0 && size(wind, 1) > 0 && size(solar, 1) > 0)
                dict[model] = sumYears(renewables) - sumYears(wind) - sumYears(solar)
            end
        elseif (variable === "Coal")
            fossilFuels = findRow(scenario, "Extraction and Conversion - Fossil Fuels", region, model)
            if (size(fossilFuels, 1) > 0)
                dict[model] = sumYears(fossilFuels) * 0.1
            end
        elseif (variable === "Oil and Gas")
            fossilFuels = findRow(scenario, "Extraction and Conversion - Fossil Fuels", region, model)
            if (size(fossilFuels, 1) > 0)
                dict[model] = sumYears(fossilFuels) * 0.9
            end
        else # End of the special cases
            row = findRow(scenario, variable, region, model)
            # If the function could find a row, we process it
            if (size(row, 1) > 0)
                dict[model] = sumYears(row)
            end
        end
    end
    dict["average"] = round(mean(values(dict)), digits=3) # We calculate the average by simply getting the mean of all values
    dict["max"] = maximum(values(dict)) # We calculate the maximum value from all values
    return dict
end

getValues (generic function with 1 method)

### Filter function
The filter function takes two parameter: a function returning true or false for each item; and a list of items.

In [28]:
function findRow(scenario, variable, region, model)
    return filter(row -> row[:Scenario] == scenario && row[:Region] == region && row[:Variable] == variable && row[:Model] == model, datum)
end

findRow (generic function with 1 method)

### Calculating the change

Function to calculate the change between two values.
We calculate the absolute change and if its positive or negative. This makes it easier to process later.

In [81]:
function calcChange(reference, value)
    if (reference === value)
       return (0, true) 
    end
    # change = round(abs(reference - value) / max(reference, value), digits=3)
    ref = reference === 0.0 ? 1 : reference
    change = round(abs(ref - value) / ref, digits = 3)
    isPositive = ref >= value
    # if (reference === 0.0)
    #    println("reference: ", reference, " value:", value, " change: ", change)
    # end
    return (min(200, change), isPositive)
end

calcChange (generic function with 1 method)

### Calculating the changes

This function iterates over all values, gets its refeference value and saves the change in the dictionary.

In [48]:
function calcChanges(vals, refs)
    changes = Dict{String, Any}()
    for (key, val) in vals
        ref = refs[key]
        changes[key] = calcChange(ref, val)
    end
    changes
end

calcChanges (generic function with 1 method)

## Build the final data set

Finally, we loop over all scenarios, variables and regions and calculate everything.

In [82]:
runs = []

for scenario in SCENARIOS
    for variable in VARIABLES
        if (variable !== "Electricity - Non-bio Renewables" && variable !== "Extraction and Conversion - Fossil Fuels")
            for region in REGIONS
                vals = getValues(scenario, variable, region)
                refs = getValues("CPol", variable, region)
                changes = calcChanges(vals, refs)
                push!(runs, (scenario = scenario, variable = variable, region = region, values = vals, reference = refs, changes = changes))
            end
        end
    end
end

## Export the dataset

`JSON.json` takes two arguments: a Dict and the indent.

In [83]:
open("../static/data/investments.json", "w") do io
   write(io, JSON.json(runs, 2));
end;

Test
